# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [10]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'gardenDB', 'local', 'nawewe', 'uk_food']


In [11]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [12]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [17]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [18]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65107e483e4ec50bd162d646'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [19]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [43]:
client = MongoClient('localhost', 27017)
db = client['uk_food']  
collection = db['establishments']
# Create a dictionary for the new restaurant data
new_restaurant = {
    "name": "Penang Flavours",
    "cuisine": "Halal",
    "location": "Greenwich",
    "rating": None  
}
new_restaurant

{'name': 'Penang Flavours',
 'cuisine': 'Halal',
 'location': 'Greenwich',
 'rating': None}

In [44]:
# Insert the new restaurant into the collection
inserted_id = establishments.insert_one(new_restaurant).inserted_id


In [39]:
# Check that the new restaurant was inserted
print("Inserted document ID:", inserted_id)

Inserted document ID: 65108525013b6364f135a826


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [51]:

# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
query


{'BusinessType': 'Restaurant/Cafe/Canteen'}

In [60]:
# Define the filter to find the BusinessTypeID for "Restaurant/Cafe/Canteen"
filter = {"BusinessType": "Restaurant/Cafe/Canteen"}

# Use find_one to retrieve the document that matches the filter
business_type_document = collection.find_one(filter)

# Check if the BusinessType document was found
if business_type_document:
    # Retrieve the BusinessTypeID from the document
    business_type_id = business_type_document.get("BusinessTypeID")
    if business_type_id:
        print("BusinessTypeID for 'Restaurant/Cafe/Canteen' is:", business_type_id)
    else:
        print("No BusinessTypeID found for 'Restaurant/Cafe/Canteen'.")


BusinessTypeID for 'Restaurant/Cafe/Canteen' is: 1


3. Update the new restaurant with the `BusinessTypeID` you found.

In [61]:
# Update the new restaurant with the correct BusinessTypeID
# Define the filter to find the "Penang Flavours" restaurant
filter = {"name": "Penang Flavours"}

# Define the update operation to set the correct BusinessTypeID
update = {"$set": {"BusinessTypeID": "1"}}  # Replace 'your_correct_id' with the actual BusinessTypeID

# Use update_one to update the document that matches the filter
result = collection.update_one(filter, update)

# Check if the update was successful
if result.modified_count > 0:
    print("Restaurant updated successfully.")
else:
    print("Restaurant not found or not updated.")


Restaurant updated successfully.


In [63]:
# Confirm that the new restaurant was updated
# Define the filter to find the "Penang Flavours" restaurant
filter = {"name": "Penang Flavours"}

# Use find_one to retrieve the updated document
updated_restaurant = collection.find_one(filter)

# Check if the updated restaurant document was found
if updated_restaurant:
    # Print the updated document to verify its contents
    print("Updated Restaurant Document:")
    print(updated_restaurant)
else:
    print("Restaurant 'Penang Flavours' not found in the collection.")


Updated Restaurant Document:
{'_id': ObjectId('6510847c013b6364f135a825'), 'name': 'Penang Flavours', 'cuisine': 'Halal', 'location': 'Greenwich', 'rating': None, 'BusinessTypeID': '1'}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [65]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}
count = collection.count_documents(query)
count

994

In [67]:
# Delete all documents where LocalAuthorityName is "Dover"
filter = {"LocalAuthorityName": "Dover"}
result = collection.delete_many(filter)
print("Deleted", result.deleted_count, "documents where LocalAuthorityName is 'Dover'")


Deleted 994 documents where LocalAuthorityName is 'Dover'


In [68]:
# Check if any remaining documents include Dover
# Define the query condition to find documents with LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}

# Use count_documents to count the matching documents
count = collection.count_documents(query)

# Check if there are any remaining documents that include "Dover"
if count > 0:
    print("There are remaining documents that include 'Dover'")
else:
    print("No remaining documents include 'Dover'")

No remaining documents include 'Dover'


In [70]:
# Check that other documents remain with 'find_one'
# Define a query condition to find one document that does not have LocalAuthorityName as "Dover"
non_dover_query = {"LocalAuthorityName": {"$ne": "Dover"}}

# Use find_one with the non-Dover query to retrieve a document that doesn't match the condition
non_dover_document = collection.find_one(non_dover_query)

# Check if there is at least one document that doesn't include "Dover"
if non_dover_document:
    print("There is at least one document that does not include 'Dover'")
else:
    print("All documents include 'Dover'")

There is at least one document that does not include 'Dover'


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [72]:
# Change the data type from String to Decimal for longitude and latitude
# Import the pymongo library and create a MongoClient
from pymongo import MongoClient
from decimal import Decimal

# Connect to your MongoDB instance (replace 'localhost' and '27017' with your MongoDB server details)
client = MongoClient('localhost', 27017)



# Define a query to retrieve documents that need to be updated
query = {"longitude": {"$type": "string"}, "latitude": {"$type": "string"}}

# Use find to retrieve the documents that match the query
documents_to_update = collection.find(query)

# Iterate through the documents and update the data types
for doc in documents_to_update:
    # Convert the "longitude" and "latitude" fields to Decimal
    doc['longitude'] = Decimal(doc['longitude'])
    doc['latitude'] = Decimal(doc['latitude'])
    
    # Update the document in the collection
    collection.replace_one({"_id": doc["_id"]}, doc)



Use `update_many` to convert `RatingValue` to integer numbers.

In [73]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [75]:
# Change the data type from String to Integer for RatingValue
# Define a query to retrieve documents that need to be updated
query = {"RatingValue": {"$type": "string"}}

# Use find to retrieve the documents that match the query
documents_to_update = collection.find(query)

# Iterate through the documents and update the data type
for doc in documents_to_update:
    # Convert the "RatingValue" field to an integer
    try:
        doc['RatingValue'] = int(doc['RatingValue'])
    except ValueError:
        # Handle cases where the conversion to an integer fails
        doc['RatingValue'] = None
    
    # Update the document in the collection
    collection.replace_one({"_id": doc["_id"]}, doc)
    

In [77]:
# Check that the coordinates and rating value are now numbers
